In [215]:
from ipyleaflet import (
    Map, GeoJSON, basemaps, basemap_to_tiles,
    LayersControl, SplitMapControl, DrawControl, MeasureControl
)
import ipywidgets as widgets
import pandas as pd
from IPython.display import display
import json
import os

In [216]:
def format_file_size(fsize):
    result = []
    units = {s: u for s, u in zip(reversed([2 ** n for n in range(0, 40, 10)]), ['GiB', 'MiB', 'KiB', 'bytes'])}
    for s, u in units.items():
        t = fsize // s
        if t > 0:
            result.append('{} {}'.format(t, u))
        fsize = fsize % s
    return ', '.join(result) or '0 bytes'

def file_path_to_size(file_path):
    return 'tamaño: ' + format_file_size(os.stat(file_path).st_size)

def new_remove(l, e):
    if e in l:
        l.remove(e)
        return l
    else: return l

In [217]:
base_path = os.path.join('/','srv','data')
folders = ['azero','guadalquivir','superficial']

geo_paths = {folder:os.path.join(base_path,folder,'geo') for folder in folders}
geo_file_lists = {folder:new_remove(os.listdir(geo_paths[folder]),'.ipynb_checkpoints') for folder in folders}
layer_names = {folder:[file[:-5] for file in geo_file_lists[folder]] for folder in folders}
layer_paths = {folder:[os.path.join(geo_paths[folder], file) for file in geo_file_lists[folder]] for folder in folders}
name_to_path = {folder:dict(zip(layer_names[folder], layer_paths[folder])) for folder in folders}

In [218]:
num_layers = 3
color_list = ['blue', 'green', 'yellow'] * 3

folder_toggle = widgets.ToggleButtons(
    options=folders,
    value=folders[0]
)

layer_dropdowns = [widgets.Dropdown(
    options=layer_names[folder_toggle.value],
    description=f'Capa {i}',
) for i in range(num_layers)]

color_pickers = [widgets.ColorPicker(
    concise=True,
    value=color,
    layout=dict(width='40px')
) for color in color_list[:num_layers]]

size_labels = [widgets.Label(
    value=file_path_to_size(name_to_path[folder_toggle.value][layer_dropdowns[i].value])
) for i in range(num_layers)]

accordion = widgets.Accordion(children=[
    folder_toggle,
    widgets.VBox([widgets.HBox([
        layer_dropdowns[i],
        color_pickers[i],
        size_labels[i]
    ]) for i in range(num_layers)])
])
accordion.set_title(0,'Datos')
accordion.set_title(1,'Capas')

go_button = widgets.Button(
    description='GO!',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Mostrar capa',
)

loading_label = widgets.Label(
    value = 'some'
)

my_map = Map(
    center=(-21.5119,-64.7253),
    zoom=7,
)
left_layer = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
right_layer = basemap_to_tiles(basemaps.Esri.WorldImagery)
split_control = SplitMapControl(left_layer=left_layer, right_layer=right_layer)
my_map.add_control(split_control)
my_map.add_control(LayersControl())
my_map.add_control(DrawControl())
my_map.add_control(MeasureControl())

output_table = widgets.Output()

In [219]:
def handle_layer_change(i):
    def handle_layer_i_change(change):
        file_path = name_to_path[folder_toggle.value][change.new]
        size_labels[i].value = file_path_to_size(file_path)
    return handle_layer_i_change

def handle_folder_change(change):
    for i in range(num_layers):
        layer_dropdowns[i].options = layer_names[change.new]

def click_handler(event=None, id=None, properties=None):
    output_table.clear_output()
    props = properties
    props.pop('style', None)
    with output_table:
        display(pd.DataFrame(props, index=[0]))

def handle_go_click(change):
    loading_label.value = 'Cargando Capas ...'
    my_map.clear_layers()
    
    ln = [layer_dropdowns[i].value for i in range(num_layers)]
    lc = [color_pickers[i].value for i in range(num_layers)]
    
    lns = []
    lcs = []
    for layer_n, layer_c in zip(ln, lc):
        if not layer_n in lns:
            lns.append(layer_n)
            lcs.append(layer_c)

    data_list = []
    
    for layer_n in lns:
        encoding = 'ISO-8859-1' if layer_n == 'predios_proceso_saneamiento' else 'utf-8'
        with open(name_to_path[folder_toggle.value][layer_n], 'r', encoding=encoding) as f:
                data_list.append(json.load(f))
        
    for data, color in zip(data_list, lcs):
        for feature in data['features']:
            feature['properties']['style'] = dict(
                color=color,
                weight=1,
                fillColor=color,
                fillOpacity=.3
            )
        
    selected_layers = [GeoJSON(
        data=data,
        name=layer_n,
        hover_style={'fillColor':'red'}
    ) for data, layer_n in zip(data_list, lns)]
    
    for selected_layer in selected_layers:
        my_map.add_layer(selected_layer)
        selected_layer.on_click(click_handler)
    
    loading_label.value = 'Listo!'

for i in range(num_layers):
    layer_dropdowns[i].observe(handle_layer_change(i),names='value')
    
folder_toggle.observe(handle_folder_change, names='value')
go_button.on_click(handle_go_click)

In [220]:
widgets.VBox([accordion, widgets.HBox([go_button, loading_label]), my_map, output_table,])